In [19]:
from transformers import pipeline
from collections import defaultdict

# Replace this with your own checkpoint
model_checkpoint = "../../../models/model_from_2023-10-30/roberta-base_16:14:29/epoch_30"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple"
)

In [20]:
def preprocess_description(description):
    single_description = description.strip()
    new_description = []
    last_special = -1
    for idx, letter in enumerate(single_description):
        if not (('a' <= letter and letter <= 'z') or ('A' <= letter and letter <= 'Z') or ('0' <= letter and letter <= '9') or letter == ' '):
            pretext = single_description[last_special + 1:idx].strip()
            if pretext != '' and pretext != ' ':
                new_description.append(pretext)
            new_description.append(letter.strip())
            last_special = idx
        if idx == len(single_description) - 1:
            new_description.append(
                single_description[last_special + 1:idx + 1].strip())
    return " ".join(new_description)

In [22]:
import json

# Replace this description
description = '''
Unlike other mixed multi - layer memory foam pads that are used to cut corners , our solid memory foam pads are superior in quality and value .
'''

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        print(bullet_point)

        results = token_classifier(bullet_point)
        for res in results:
            if res['word'].lower().strip() in ['durable', 'strong', 'heavy-duty', 'heavy duty', 'stability', 'versatile']:
                continue
            group = res['entity_group']
            if res['score'] >= 0.6:
                high_score_ans[group].add(res['word'].lower().strip())
                
new_high_score_ans = defaultdict(list)
for key_dict in high_score_ans.keys():
    new_high_score_ans[key_dict] = list(high_score_ans[key_dict])

print("-"*100)            
print(json.dumps(new_high_score_ans, sort_keys=True, indent=4))                

Unlike other mixed multi - layer memory foam pads that are used to cut corners , our solid memory foam pads are superior in quality and value . 
----------------------------------------------------------------------------------------------------
{
    "MAT": [
        "solid memory foam"
    ],
    "NMAT": [
        "memory foam"
    ],
    "PROPERTY": [
        "multi - layer"
    ]
}
